In [ ]:
def L_i(x, y, W):
  """
  L_i: versión sin vectorizar (para un solo ejemplo).
  unvectorized version. Compute the multiclass svm loss for a single example (x,y)
  - x is a column vector representing an image (e.g. 3073 x 1 in CIFAR-10)
    with an appended bias dimension in the 3073-rd position (i.e. bias trick)
  - y is an integer giving index of correct class (e.g. between 0 and 9 in CIFAR-10)
  - W is the weight matrix (e.g. 10 x 3073 in CIFAR-10)
  """
  delta = 1.0 # see notes about delta later in this section
  scores = W.dot(x) # scores becomes of size 10 x 1, the scores for each class
  correct_class_score = scores[y]
  D = W.shape[0] # number of classes, e.g. 10
  loss_i = 0.0
  for j in range(D): # iterate over all wrong classes
    if j == y:
      # skip for the true class to only loop over incorrect classes
      continue
    # accumulate loss for the i-th example
    loss_i += max(0, scores[j] - correct_class_score + delta)
  return loss_i

def L_i_vectorized(x, y, W):
  """
  L_i_vectorized: versión medio vectorizada (aún solo para un ejemplo).
  A faster half-vectorized implementation. half-vectorized
  refers to the fact that for a single example the implementation contains
  no for loops, but there is still one loop over the examples (outside this function)
  """
  delta = 1.0
  scores = W.dot(x)
  # compute the margins for all classes in one vector operation
  margins = np.maximum(0, scores - scores[y] + delta)
  # on y-th position scores[y] - scores[y] canceled and gave delta. We want
  # to ignore the y-th position and only consider margin on max wrong class
  margins[y] = 0
  loss_i = np.sum(margins)
  return loss_i

def L(X, y, W):
  """
  Fully-vectorized SVM loss.
  - X: matriz de datos (N x D), cada fila es un ejemplo
  - y: vector de etiquetas correctas (N,)
  - W: matriz de pesos (C x D)
  Retorna:
  - pérdida escalar (float)
  """

  delta = 1.0
  scores = X.dot(W.T)  # (N x C): cada fila contiene scores por clase
  N = X.shape[0]

  # Obtener score correcto para cada fila (N x 1)
  correct_class_scores = scores[np.arange(N), y].reshape(-1, 1)

  # Calcular márgenes
  margins = np.maximum(0, scores - correct_class_scores + delta)
  margins[np.arange(N), y] = 0  # Ignorar la clase correcta

  # Pérdida total promedio
  loss = np.sum(margins) / N
  return loss


In [ ]:
import numpy as np

# Simular pesos (3 clases, 5 características)
W = np.array([
  [0.1, 0.2, 0.3, 0.4, 0.5],
  [0.5, 0.4, 0.3, 0.2, 0.1],
  [0.2, 0.3, 0.4, 0.5, 0.6]
])

# Simular 4 ejemplos de entrada
X = np.array([
  [1, 0, 0, 0, 1],
  [0, 1, 0, 1, 0],
  [1, 1, 1, 0, 0],
  [0, 0, 0, 1, 1]
])

# Etiquetas correctas
y = np.array([0, 1, 2, 1])

# Calcular pérdida
print("Loss:", L(X, y, W))


Loss: 2.45


In [ ]:
def svm_loss(X, y, W, delta=1.0):
    N = X.shape[0]
    scores = X.dot(W.T)  # (N x C)
    correct_scores = scores[np.arange(N), y].reshape(-1, 1)
    margins = np.maximum(0, scores - correct_scores + delta)
    margins[np.arange(N), y] = 0
    loss = np.sum(margins) / N
    return loss


In [ ]:
def softmax_loss(X, y, W):
    N = X.shape[0]
    scores = X.dot(W.T)

    # Evitar explosión exponencial (trick numérico)
    scores -= np.max(scores, axis=1, keepdims=True)

    exp_scores = np.exp(scores)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    correct_logprobs = -np.log(probs[np.arange(N), y])
    loss = np.sum(correct_logprobs) / N
    return loss


In [ ]:
svm = svm_loss(X, y, W)
softmax = softmax_loss(X, y, W)

print(f"Pérdida SVM:     {svm:.4f}")
print(f"Pérdida Softmax: {softmax:.4f}")


Pérdida SVM:     2.4500
Pérdida Softmax: 1.2717
